In [8]:
import pandas as pd
import numpy as np
import sqlite3
import json

In [39]:
with sqlite3.connect("data/storage.db") as conn:
    fdf = pd.read_sql_query('SELECT id, wall_id, angle, holds, grade, quality, ascents FROM climbs WHERE ascents > 1', conn, index_col="id")
    fdf['holds']=fdf['holds'].apply(json.loads)
    fdf = fdf[fdf['holds'].apply(lambda h: len(h)) <= 20]

    hdf = pd.read_sql_query("SELECT hold_index, x, y, pull_x, pull_y, useability, is_foot FROM holds WHERE wall_id LIKE 'wall-443c15cd12e0'", conn, index_col='hold_index')

In [40]:
hdf.head()

,x,y,pull_x,pull_y,useability,is_foot
hold_index,,,,,,
390,0.666667,0.333333,0.0,-1.0,0.5,1
392,0.666667,1.666667,0.0,-1.0,0.5,1
394,0.666667,3.000000,0.0,-1.0,0.5,0
396,0.666667,4.333333,0.0,-1.0,0.5,0
399,0.666667,6.333333,0.0,-1.0,0.5,0


In [30]:
fdf.head()

,wall_id,angle,holds,grade,quality,ascents
id,,,,,,
climb-5f68ab77af17,wall-443c15cd12e0,40,"[[570, 0], [512, 1], [523, 2], [576, 2], [578,...",15.9936,2.97925,5011
climb-89a4f1804603,wall-443c15cd12e0,40,"[[534, 0], [607, 0], [687, 1], [504, 2], [611,...",18.0034,2.98120,4095
climb-a7649d8bb38b,wall-443c15cd12e0,40,"[[699, 0], [684, 1], [691, 1], [747, 2], [787,...",15.9897,2.96407,3896
climb-b0df21d76511,wall-443c15cd12e0,40,"[[641, 0], [747, 0], [792, 1], [680, 2], [752,...",13.0074,2.97686,3673
climb-ae7536aebc95,wall-443c15cd12e0,40,"[[734, 0], [582, 1], [615, 2], [684, 2], [619,...",15.0011,2.98169,3660
